# A Tutorial on Function Calling with Gemma 3

Function calling is an incredibly powerful construct that allows large language models like Gemma to use tools and interact with external APIs. It empowers Gemma to go beyond just generating text and actually do real world tasks for you, like checking the weather, booking a flight, keeping track of your expenses, and much much more! Combined with Gemma's multimodal capabilities and offline nature, function calling opens up a lot of amazing possibilities.

This tutorial will introduce you to function calling with Gemma 3 through a real world example - we will use Gemma to keep track of our expenses and stay under a monthly budget.

## Introduction

Before we start writing code, let us understand how function calling works:

![flowchart](media/flowchart.svg)

1. We inform a model that it has access to a given set of functions, and specify how it can invoke the functions.
2. The user asks the model to perform a task, and the model realises it has to call a function to complete the task.
3. The model includes a function call in its response - this is typically a JSON object with the name of the function it wants to call, and the parameters to pass when calling the function.
5. The program the user is using to talk to the model, parses the model's response for function calls. The functions are then executed and the output is typically sent back as a JSON object.
6. The model uses the output of the function call to produce a natural language response for the user.

Note that `toolkit` refers to the function calling toolkit used in this tutorial so that we don't have to write too much boilerplate code, and we can focus on the function calling side of things :)

## Getting Started

This tutorial assumes you have basic knowledge of Python. In addition, this tutorial uses [`ollama`](https://ollama.com) to run the models locally on your computer, and [`uv`](https://docs.astral.sh/uv/getting-started/) to run python code and manage dependencies. If you haven't already, please go to the linked pages to download and install the necessary tools on your computer.

Start Ollama by running the following command:

```bash
ollama serve
```

In another terminal, clone this repository and open the notebook by running:

```bash
git clone https://github.com/gamemaker1/gemma3-function-calling-tutorial.git
cd gemma3-function-calling-tutorial/
uv run --with jupyter jupyter lab tutorial.ipynb
```

This should open this tutorial notebook in your browser so you can follow along!

In [1]:
# (ignore) this code imports the toolkit and defines functions for rendering markdown in notebooks.

import re
import json
import toolkit

from hashlib import sha256
from datetime import datetime

from rich.console import Console
from rich.markdown import Markdown
from rich.live import Live
from rich.spinner import Spinner

console = Console(force_jupyter=True, width=100)


def render_markdown(text):
    text = re.sub(r"```function_spec\b", "```json", text)
    text = re.sub(r"```function_call\b", "```json", text)
    text = re.sub(r"```function_output\b", "```json", text)

    return Markdown(text)


def render_json(obj):
    return render_markdown(f"```json\n{json.dumps(obj, indent=2)}\n```\n")


def render_live(stream):
    response = ""

    with Live(
        Spinner("dots", text="Loading..."), refresh_per_second=10, console=console
    ) as live:
        for chunk in stream:
            response += chunk
            live.update(render_markdown(response))

    return response

### Selecting a Model

Function calling works best with the 12B and 27B parameter Gemma 3 models. If you do not have enough resources to run the 27B parameter model, please replace `gemma3:27b` with `gemma3:12b` in the following line and the `pull` command below.

In [2]:
model = "gemma3:27b"

Run the following command in terminal to download the model (this might take a while and several GB of data):

```bash
ollama pull gemma3:27b
```

### Instructing the Model

Gemma 3 is not specifically trained for function calling and [does not have a system prompt](https://ai.google.dev/gemma/docs/core/prompt-structure#system-instructions). However, providing instructions on how and what functions to use along with our messages effectively enables function calling in Gemma.

The instructions we provide for function calling might vary based on the model we are using. For the 12B and 27B parameter variants of Gemma 3, the following prompt has been observed to give great results.

In [3]:
prompt = toolkit.prompts.function_calling(model)
render_markdown(prompt)

You are a multipurpose, thinking assistant to me, the user :)                                                      

When I send you a message, I will either:                                                                          

 1 start a new thread, or                                                                                          
 2 continue an existing thread.                                                                                    

When I start a new thread, I will ask you something or assign you a task. You must either answer completely or     
complete the task for the thread to end.                                                                           

To respond to my asks or tasks, you must only do one of the following:                                             

a. ask for clarifications b. call one or more functions with some parameters c. conclude the thread with a final   
response.                                                                                                          

Think carefully about what I am asking you. Figure out my intent, and think about what courses of action you can   
take. Try them all and figure out which one works best. You must preface all your responses with the chain of      
thought that lead to your choice and response. Put this thinking at the top of your response, enclosed in a code   
block like so:                                                                                                     

                                                                                                                   
 make sure you mention which option you choose from the above (a, b, c), along with your thoughts here.            
                                                                                                                   

The functions mentioned in choice (b) will be provided in code blocks like so:                                     

                                                                                                                   
 the function specification, in json or yaml format.                                                               
                                                                                                                   

While calling functions, make sure you follow the specification of the function. Do not make assumptions or        
guesses, and feel free to ask the user for clarifications before calling functions.                                

To call a function, you must output the following code block (it must be valid JSON) in your response:             

                                                                                                                   
 {                                                                                                                 
     "id": this string will be mentioned in the function output so you know which function call produced which     
 output,                                                                                                           
     "function": the name of the function to call,                                                                 
     "parameters": a dictionary of parameter names and values to pass to the function                              
 }                                                                                                                 
                                                                                                                   

Note that you must only use the functions provided to you. You may call multiple functions in parallel by including
multiple function_call code blocks in your response when there are no data dependencies between them. The responses
to your function calls might be returned out of order, across several user messages. I encourage you to use        
parallel function calling as much as possib

Notice the following key instructions given to the model in the above prompt:

1. The concept of 'threads' is introduced to the model. This enables the model to keep better track of the context it can use while calling functions.
2. The model is given three options to use while responding to messages in a thread. This forces more deterministic behaviour from the model, and introduces the concept of function calling in option (b). The model is also asked to ensure its final responses are user-friendly and in natural language.
3. A few lines describe the thinking process that the model should follow before responding, and ask the model to output this in code blocks with the `thinking` label.
4. The rest of the prompt specifies how functions will be declared (in code blocks with the `function_spec` label), how they can be invoked (in code blocks with the `function_call` label, in parallel or in sequence), and how to deal with the returned function output (in code blocks with the `function_output` label, possibly returned out of order).
5. Gemma has a tendency to produce markdown, so the use of code blocks with labels is made to force structured output that can be parsed by programs, while also keeping the ability to communicate with the user directly using natural language.

The function specifications are valid JSON objects, following [this schema](schemas/function.schema.json). The specifications must include a detailed description of what the function does, what parameters it accepts, the possible outputs it produces, and the possible errors it throws, along with examples of prompts that should trigger its invocation. This level of detail enables the model to make best use of the functions provided to it.

### Defining the Functions

Now that we have the instructions for the model, we can write out our functions' code (in Python) and specifications (as per the schema mentioned in the prompt).

In [4]:
# (click to open) the custom errors our functions raise.


class BadParameters(Exception):
    pass


class NoResults(Exception):
    pass

In [5]:
# (click to open) the code for our expense tracking functions.

transactions = []


def make_expense_entry(
    amount: float, category: str, date: str, notes: str = None
) -> str:
    global transactions

    try:
        datetime.strptime(date, "%Y-%m-%d")
    except ValueError:
        raise BadParameters("The `date` must be in yyyy-mm-dd format.")

    entry = {
        "amount": amount,
        "category": category,
        "date": date,
        "notes": notes,
        "created": datetime.now().isoformat(),
    }

    identifier = sha256(str.encode(json.dumps(entry))).hexdigest()[:8]
    entry["id"] = identifier

    transactions += [entry]

    return identifier


def get_expense_entries(
    category: str = None, from_date: str = None, to_date: str = None
) -> dict:
    global transactions

    filtered = []
    for transaction in transactions:
        if category and transaction["category"].lower() != category.lower():
            continue
        if from_date and transaction["date"] < from_date:
            continue
        if to_date and transaction["date"] > to_date:
            continue
        filtered.append(transaction)

    if not filtered:
        raise NoResults("The search yielded no results with the given filters.")

    return {
        "transactions": filtered,
        "count": len(filtered),
        "total": sum(x["amount"] for x in filtered),
    }

In [6]:
# (click to open) the specifications for our two custom functions.

specifications = {
    "make_expense_entry": {
        "name": "make_expense_entry",
        "description": "Add an expense entry to the user's personal accounting records.",
        "parameters": {
            "type": "object",
            "properties": {
                "amount": {
                    "type": "float",
                    "description": "The amount spent by the user.",
                },
                "category": {
                    "type": "string",
                    "enum": [
                        "food",
                        "entertainment",
                        "clothing",
                        "travel",
                        "utilities",
                        "other",
                    ],
                    "description": "The category the expense falls under.",
                },
                "date": {
                    "type": "string",
                    "format": "yyyy-mm-dd",
                    "description": "The date of the expense",
                },
                "notes": {
                    "type": "string",
                    "description": "A short note about the expense, that will remind the user of it if needed.",
                },
            },
            "required": ["amount", "category", "date"],
        },
        "responses": [
            {"type": "string", "description": "The ID of the created expense entry."}
        ],
        "errors": [
            {"name": "BadEntry", "description": "The category or date are invalid."}
        ],
    },
    "get_expense_entries": {
        "name": "get_expense_entries",
        "description": "Fetches expense entries from the user's personal accounting records, filtering by category and date.",
        "parameters": {
            "type": "object",
            "properties": {
                "category": {
                    "type": "string",
                    "enum": [
                        "food",
                        "entertainment",
                        "clothing",
                        "travel",
                        "utilities",
                        "other",
                    ],
                    "description": "The category the expense falls under, if filtering by category.",
                },
                "from_date": {
                    "type": "string",
                    "format": "yyyy-mm-dd",
                    "description": "If filtering by date, this is the start of the search period (inclusive).",
                },
                "to_date": {
                    "type": "string",
                    "format": "yyyy-mm-dd",
                    "description": "If filtering by date, this is the end of the search period (inclusive).",
                },
            },
            "required": [],
        },
        "responses": [
            {
                "type": "object",
                "properties": {
                    "entries": {
                        "type": "array",
                        "items": {
                            "type": "object",
                            "properties": {
                                "amount": {
                                    "type": "float",
                                    "description": "The amount spent by the user.",
                                },
                                "category": {
                                    "type": "string",
                                    "enum": [
                                        "food",
                                        "entertainment",
                                        "clothing",
                                        "travel",
                                        "utilities",
                                        "other",
                                    ],
                                    "description": "The category the expense falls under.",
                                },
                                "notes": {
                                    "type": "string",
                                    "description": "A short note about the expense, that will remind the user of it if needed.",
                                },
                                "date": {
                                    "type": "string",
                                    "format": "yyyy-mm-dd",
                                    "description": "The date of the expense",
                                },
                                "created": {
                                    "type": "string",
                                    "format": "iso8601",
                                    "description": "The timestamp recording when the entry was made.",
                                },
                                "id": {
                                    "type": "string",
                                    "description": "The ID of the expense entry.",
                                },
                            },
                        },
                    },
                    "count": {
                        "type": "int",
                        "description": "The number of transactions returned.",
                    },
                    "total": {
                        "type": "float",
                        "description": "The sum total of the amount spent in the transactions returned.",
                    },
                },
            }
        ],
        "errors": [
            {
                "name": "BadEntry",
                "description": "The category, from date or to date are invalid.",
            },
            {
                "name": "NoResults",
                "description": "The search yielded no results with the given filters.",
            },
        ],
    },
}

In the above blocks is the code and the specification for two functions:

1. `make_expense_entry`: allows us to enter the amount spent, which category the expense belongs to, the date of the transaction, and a short note about the expense (optional).
2. `get_expense_entries`: allows us to fetch all the expenses we have entered so far, and optionally filter them based on category or date.

Next, we register the function code and specification with the toolkit so that we can send the model the function specifications, and call the functions when they are invoked by the model.

In [7]:
toolkit.functions.register(make_expense_entry, specifications["make_expense_entry"])
toolkit.functions.register(get_expense_entries, specifications["get_expense_entries"])

Let's try calling the functions ourselves in Python first:

In [8]:
make_expense_entry(78, "travel", "2025-05-19")
make_expense_entry(231.5, "food", "2025-05-19")
make_expense_entry(459, "food", "2025-05-20")
make_expense_entry(917, "clothing", "2025-05-20")

render_json(get_expense_entries())

                                                                                                                   
 {                                                                                                                 
   "transactions": [                                                                                               
     {                                                                                                             
       "amount": 78,                                                                                               
       "category": "travel",                                                                                       
       "date": "2025-05-19",                                                                                       
       "notes": null,                                                                                              
       "created": "2025-06-15T18:52:31.895795",                                                                    
       "id": "c9b938ab"                                                                                            
     },                                                                                                            
     {                                                                                                             
       "amount": 231.5,                                                                                            
       "category": "food",                                                                                         
       "date": "2025-05-19",                                                                                       
       "notes": null,                                                                                              
       "created": "2025-06-15T18:52:31.895954",                                                                    
       "id": "515aabbb"                                                                                            
     },                                                                                                            
     {                                                                                                             
       "amount": 459,                                                                                              
       "category": "food",                                                                                         
       "date": "2025-05-20",                                                                                       
       "notes": null,                                                                                              
       "created": "2025-06-15T18:52:31.896029",                                                                    
       "id": "ee65ff21"                                                                                            
     },                                                                                                            
     {                                                                                                             
       "amount": 917,                                                                                              
       "category": "clothing",                                                                                     
       "date": "2025-05-20",                                                                                       
       "notes": null,                                                                                              
       "created": "2025-06-15T18:52:31.896089",                                                                    
       "id": "61832ee0"                                                                                            
     }                                                  

It works! Our expenses are being stored whenever the `make_expense_entry` function is called, and a list of expense entries is returned whenever the `get_expense_entries` function is called.

The next thing we need to do is collates the function specifications into code blocks with the `function_spec` label, since that is the format the model will expect it in. The toolkit provides a helper function that does this for all registered functions:

In [9]:
functions = toolkit.functions.specify()
render_markdown(functions)

                                                                                                                   
 {                                                                                                                 
   "name": "make_expense_entry",                                                                                   
   "description": "Add an expense entry to the user's personal accounting records.",                               
   "parameters": {                                                                                                 
     "type": "object",                                                                                             
     "properties": {                                                                                               
       "amount": {                                                                                                 
         "type": "float",                                                                                          
         "description": "The amount spent by the user."                                                            
       },                                                                                                          
       "category": {                                                                                               
         "type": "string",                                                                                         
         "enum": [                                                                                                 
           "food",                                                                                                 
           "entertainment",                                                                                        
           "clothing",                                                                                             
           "travel",                                                                                               
           "utilities",                                                                                            
           "other"                                                                                                 
         ],                                                                                                        
         "description": "The category the expense falls under."                                                    
       },                                                                                                          
       "date": {                                                                                                   
         "type": "string",                                                                                         
         "format": "yyyy-mm-dd",                                                                                   
         "description": "The date of the expense"                                                                  
       },                                                                                                          
       "notes": {                                                                                                  
         "type": "string",                                                                                         
         "description": "A short note about the expense, that will remind the user of it if needed."               
       }                                                                                                           
     },                                                                                                            
     "required": [                                                                                                 
       "amount",                                        

### Calling the Function

<div style="display: flex; align-items: flex-start; gap: 20px;">
    <div style="flex: 1;">
        Now that we have the prompt and the function specifications, we can ask the model whatever we want to. Say we just ate at a restaurant, and got a bill (shown on the right).<br><br>
        We can give this image to Gemma, and ask it to add this to our expenses. We can even ask it to check if we're going over our monthly budget for food!<br><br>
        All we have to do is setup the conversation - the first message must be the function calling prompt, the second must be the specifications of the functions that Gemma can call, and the third must be our task for the model. 
    </div>
    <div style="flex-shrink: 0;">
        <img src="media/bill.jpg" width="400" height="100" />
    </div>
</div>

In [10]:
task = "Add this to my expenses, and let me know if I'm going over my food budget of 1000 this month."

In [11]:
messages = [
    {"role": "user", "content": prompt},
    {"role": "user", "content": functions},
    {"role": "user", "content": task, "images": ["media/bill.jpg"]},
]

The toolkit provides a helper function to chat with the model, which supports sending images and streaming responses. Use it to get the model's response and add that to the conversation.

In [12]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

Output()

The model's response contains two code blocks - the first with a `thinking` label, and the second with a `function_call` label. The first code block contains its thoughts on how to complete the task. The second code block contains the function to call, and the parameters to pass to it. It also contains an `id` field, which we must return along with the output of the function call. This lets the model know which function call produced the output it is receiving.

Notice how the model intelligently extracts parameters from the information available to it - it gets the amount, category, and date of the expense all from the bill we provided to it. It even assumes the current month is May, since the receipt is dated May 2025. 

Now that we have the complete model response, we need to parse it for function calls. The toolkit provides a function that extracts JSON contents of code blocks labelled `function_call`.

In [13]:
calls = toolkit.functions.parse_calls(complete_response)
render_json(calls)

                                                                                                                   
 [                                                                                                                 
   {                                                                                                               
     "id": "1",                                                                                                    
     "function": "make_expense_entry",                                                                             
     "parameters": {                                                                                               
       "amount": 588.0,                                                                                            
       "category": "food",                                                                                         
       "date": "2025-05-04",                                                                                       
       "notes": "Dinner at TSC"                                                                                    
     }                                                                                                             
   }                                                                                                               
 ]                                                                                                                 
                                                                                                                   


We can now loop through these function calls and execute them one by one using a helper function from the toolkit. It identifies the function to execute from the list of registered functions, invokes the function with the given named parameters, and formats the function's output in JSON.

Since functions can be asynchronous, the results are passed via a callback. Our callback, the `respond` function, simply stores the results of the function call as a message from the user.

In [14]:
def respond(result: str) -> None:
    global messages
    messages.append({"role": "user", "content": result})


for call in calls:
    toolkit.functions.execute_call(call, respond)

Once the above code finishes executing, the contents of the last message will be the output of the function.

In [15]:
render_markdown(messages[-1]["content"])

                                                                                                                   
 {                                                                                                                 
     "id": "1",                                                                                                    
     "result": "0b87f506"                                                                                          
 }                                                                                                                 
                                                                                                                   


Notice that the output is wrapped in a JSON object, with the `id` being the same as the `id` received in the function call. This lets the model know which function call produced the output it is receiving.

Now that we have the function output, we can continue the conversation with the model:

In [16]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

Output()

We get two code blocks again - one with the model's thoughts, and the other with another function call. Notice how it acknowledges the output of the previous function - it knows the call is successful, and that the returned string is the ID of the expense entry. This is thanks to the detailed function specification we gave it.

Now, let us parse and execute the new function call and inspect its result:

In [17]:
calls = toolkit.functions.parse_calls(complete_response)
for call in calls:
    toolkit.functions.execute_call(call, respond)

In [18]:
render_markdown(messages[-1]["content"])

                                                                                                                   
 {                                                                                                                 
     "id": "2",                                                                                                    
     "result": {                                                                                                   
         "transactions": [                                                                                         
             {                                                                                                     
                 "amount": 231.5,                                                                                  
                 "category": "food",                                                                               
                 "date": "2025-05-19",                                                                             
                 "notes": null,                                                                                    
                 "created": "2025-06-15T18:52:31.895954",                                                          
                 "id": "515aabbb"                                                                                  
             },                                                                                                    
             {                                                                                                     
                 "amount": 459,                                                                                    
                 "category": "food",                                                                               
                 "date": "2025-05-20",                                                                             
                 "notes": null,                                                                                    
                 "created": "2025-06-15T18:52:31.896029",                                                          
                 "id": "ee65ff21"                                                                                  
             },                                                                                                    
             {                                                                                                     
                 "amount": 588.0,                                                                                  
                 "category": "food",                                                                               
                 "date": "2025-05-04",                                                                             
                 "notes": "Dinner at TSC",                                                                         
                 "created": "2025-06-15T18:53:30.359511",                                                          
                 "id": "0b87f506"                                                                                  
             }                                                                                                     
         ],                                                                                                        
         "count": 3,                                                                                               
         "total": 1278.5                                                                                           
     }                                                                                                             
 }                                                                                                                 
                                                        

Nice! We see that the `get_expense_entries` function works as intended, returning all the expense entries in the `food` category for the month of May. Let's send this output back to the model and get its response:

In [19]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

Output()

Nice! The model acknowledged the output of the function call, and used it to generate a response to our question of whether we had exceeded our food budget or not. This completes the task - and your first function calling enabled conversation with Gemma!

Next, let's try parallel function calling by asking the model to check if we've exceeded our clothes and travel budgets too. Since the two function calls the model needs to make are independent of each other, the model _should_ make them in parallel, i.e., produce two `function_call` labelled code blocks in a single response.

In [20]:
task = "Oh no! Could you also check if I have gone above my clothes and travel budgets?"
messages.append({"role": "user", "content": task})

In [21]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

Output()

Ah! Instead of calling the functions, the model is requesting a clarification. This is because we haven't mentioned our budget for the clothing or travel category, and the model has rightly not made any assumptions. Let us respond with a clarification:

In [22]:
clarification = "My clothing budget is 500, and my travel budget is 2k."
messages.append({"role": "user", "content": clarification})

In [23]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

Output()

Nice! It generated two calls to the `get_expense_entries` function with different IDs - passing 'clothing' as the category in the first one, and 'travel' as the category in the second. Let's parse and execute these function calls.

In [24]:
calls = toolkit.functions.parse_calls(complete_response)
for call in calls:
    toolkit.functions.execute_call(call, respond)

The successful execution of the functions means two messages have been added to the conversation:

In [25]:
render_markdown(messages[-2]["content"])

                                                                                                                   
 {                                                                                                                 
     "id": "3",                                                                                                    
     "result": {                                                                                                   
         "transactions": [                                                                                         
             {                                                                                                     
                 "amount": 917,                                                                                    
                 "category": "clothing",                                                                           
                 "date": "2025-05-20",                                                                             
                 "notes": null,                                                                                    
                 "created": "2025-06-15T18:52:31.896089",                                                          
                 "id": "61832ee0"                                                                                  
             }                                                                                                     
         ],                                                                                                        
         "count": 1,                                                                                               
         "total": 917                                                                                              
     }                                                                                                             
 }                                                                                                                 
                                                                                                                   


In [26]:
render_markdown(messages[-1]["content"])

                                                                                                                   
 {                                                                                                                 
     "id": "4",                                                                                                    
     "result": {                                                                                                   
         "transactions": [                                                                                         
             {                                                                                                     
                 "amount": 78,                                                                                     
                 "category": "travel",                                                                             
                 "date": "2025-05-19",                                                                             
                 "notes": null,                                                                                    
                 "created": "2025-06-15T18:52:31.895795",                                                          
                 "id": "c9b938ab"                                                                                  
             }                                                                                                     
         ],                                                                                                        
         "count": 1,                                                                                               
         "total": 78                                                                                               
     }                                                                                                             
 }                                                                                                                 
                                                                                                                   


Now that we have the outputs to both function calls, we can send it to the model and get its final response.

In [27]:
streaming_response = toolkit.model.get_response(model, messages)
complete_response = render_live(streaming_response)
messages.append({"role": "assistant", "content": complete_response})

Output()

Nice! The model acknowledges both the outputs and uses them to construct a user-friendly response, answering our question about the clothing and travel budgets.